In [2]:
#Limpiar la zona de trabajo
rm(list=ls())
#Librerias
if (require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,
               readr, 
               srvyr,
               ggplot2,
               showtext, 
               dplyr, 
               purrr)

Warning message:
"package 'pacman' is in use and will not be installed"


In [3]:
#Consideraciones----
#Tiempo en linea
options(timeout = 8000)

In [10]:
# Función para leer los datos de los censos
leer_ce <- function(anios) { 
  
  anios <- as.integer(anios)[anios %in% c(2004, 2009, 2014, 2019)]
  data_list <- list()
  
  for (anio in anios) {
    
    url <- paste0("https://www.inegi.org.mx/contenidos/programas/ce/2019/Datosabiertos/ce",
                  anio,
                  "_cdmx_csv.zip")
    
    temp_zip <- tempfile(fileext = ".zip")
    temp_dir <- tempdir()
    options(timeout = 9999)
    
    # Descargar y descomprimir ZIP
    download.file(url, destfile = temp_zip, mode = "wb", quiet = TRUE)
    unzip(temp_zip, exdir = temp_dir)
    
    # Construir nombre del archivo esperado
    nombre_csv <- file.path(temp_dir, "conjunto_de_datos", paste0("ce", anio, "_cdmx.csv"))
    
    # Leer CSV
    datos <- readr::read_csv(nombre_csv,
                             col_types = readr::cols(),
                             show_col_types = FALSE) |>
      janitor::clean_names() |>
      dplyr::mutate(anio = as.integer(anio))
    
    data_list[[as.character(anio)]] <- datos
  }
  
  return(data_list)
}


In [12]:
#Leer datos de los censos económicos
lista_censos <- leer_ce(c(2004, 2009, 2014, 2019))
names(lista_censos)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] "2004" "2009" "2014" "2019"

In [19]:
head(lista_censos$`2019`)

entidad,municipio,codigo,id_estrato,ue,a111a,a121a,a131a,a211a,a221a,⋯,a433a,a434a,a436a,a529a,a534a,a747a,a748a,a752a,a764a,anio
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
09,NA,NA,NA,427959,4280286.8,1987650.3,2292636.5,247810.900,236511.907,⋯,153.595,85.842,1306291.2,7.696,23.824,397.955,748.773,67.858,"40.939,",2019
09,NA,NA,1,396040,503719.3,159108.5,344610.7,5776.747,6230.130,⋯,218.602,88.443,565698.0,7.847,42.859,207.241,312.167,222.481,"44.703,",2019
09,NA,NA,2,23622,381732.2,190345.6,191386.6,4808.947,6071.708,⋯,149.732,88.033,1270632.4,8.730,31.935,243.101,383.557,104.845,"42.579,",2019
09,NA,NA,3,6441,713094.4,401344.7,311749.7,22670.862,11177.463,⋯,130.634,80.859,1717864.2,8.016,24.903,317.737,491.638,110.039,"39.898,",2019
09,NA,NA,4,1856,2681740.9,1236851.5,1444889.5,214554.344,213032.606,⋯,153.373,86.280,1504924.7,7.587,20.184,453.951,809.580,52.392,"39.244,",2019
09,NA,11,NA,24,NA,NA,NA,NA,NA,⋯,256.393,97.024,321823.5,15.221,35.746,330.476,190.388,185.547,"24.510,",2019


In [30]:
total_unidades <- purrr::imap_dfr(lista_censos, ~ .x |>
  dplyr::select(id_estrato = id_estrato, codigo = codigo, municipio = municipio, ue=ue) |>
  dplyr::mutate(anio = as.integer(.y)))|>
  dplyr::filter(is.na(id_estrato) & is.na(codigo) & is.na(municipio))|>
  select(anio, ue)

total_unidades

anio,ue
<int>,<dbl>
2004,342475
2009,382056
2014,415481
2019,427959


In [40]:
#Actividades economía circular
actividades <- data.frame(actividad = c("Comercio al por mayor de materiales de desecho",
                                "Comercio al por menor de artículos usados",
                                "Manejo de residuos y servicios de remediación",
                                "Restauradores de muebles antiguos",
                                "Reparación y mantenimiento de automóviles y camiones",
                                "Reparación y mantenimiento de equipo electrónico y de equipo de precisión",
                                "Reparación y mantenimiento de artículos para el hogar y personales",
                                "Alquiler de uniformes, blancos y pañales",
                                "Remanufactura de cartuchos de tóner",
                                "Alquiler de automóviles, camiones y otros transportes terrestres",
                                "Alquiler de artículos para el hogar y personales",
                                "Centros generales de alquiler", 
                                "Alquiles de equipo de cómputo y otras maquinas y mobiliario de oficina"),
                                clave = c("43431", "46641", "562", "71151", "81111", "81121", "8114",
                                "81221", "3251","53211", "53228", "53231", "53242"),
                                stringsAsFactors = FALSE)


In [46]:
colnames(lista_censos$`2019`)
str(lista_censos$`2019`)

[1] "entidad"    "municipio"  "codigo"     "id_estrato" "ue"        
  [6] "a111a"      "a121a"      "a131a"      "a211a"      "a221a"     
 [11] "a511a"      "a700a"      "a800a"      "h000a"      "h000b"     
 [16] "h000c"      "h000d"      "h001a"      "h001b"      "h001c"     
 [21] "h001d"      "h010a"      "h010b"      "h010c"      "h010d"     
 [26] "h020a"      "h020b"      "h020c"      "h020d"      "h101a"     
 [31] "h101b"      "h101c"      "h101d"      "h203a"      "h203b"     
 [36] "h203c"      "h203d"      "i000a"      "i000b"      "i000c"     
 [41] "i000d"      "i100a"      "i100b"      "i100c"      "i100d"     
 [46] "i200a"      "i200b"      "i200c"      "i200d"      "j000a"     
 [51] "j010a"      "j203a"      "j300a"      "j400a"      "j500a"     
 [56] "j600a"      "k000a"      "k010a"      "k020a"      "k030a"     
 [61] "k042a"      "k412a"      "k050a"      "k060a"      "k070a"     
 [66] "k090a"      "k096a"      "k311a"      "k610a"      "k620a"     
 [71] "k810a"      "k820a"      "k910a"      "k950a"      "k976a"     
 [76] "m000a"      "m010a"      "m020a"      "m030a"      "m050a"     
 [81] "m090a"      "m091a"      "m700a"      "o010a"      "o020a"     
 [86] "p000a"      "p000b"      "p000c"      "p030a"      "p030b"     
 [91] "p030c"      "p100a"      "p100b"      "q000a"      "q000b"     
 [96] "q000c"      "q000d"      "q010a"      "q020a"      "q030a"     
[101] "q400a"      "q900a"      "a112a"      "a113a"      "a114a"     
[106] "a115a"      "a116a"      "a117a"      "a118a"      "a119a"     
[111] "a141a"      "a143a"      "a144a"      "a145a"      "a146a"     
[116] "a147a"      "a148a"      "a149a"      "a151a"      "a152a"     
[121] "a153a"      "a154a"      "a155a"      "a156a"      "a157a"     
[126] "a158a"      "a159a"      "a171a"      "a173a"      "a174a"     
[131] "a175a"      "a176a"      "a177a"      "a179a"      "a181a"     
[136] "a182a"      "a192a"      "a193a"      "a194a"      "a195a"     
[141] "a197a"      "a201a"      "a202a"      "a204a"      "a206a"     
[146] "a208a"      "a209a"      "a212a"      "a213a"      "a214a"     
[151] "a215a"      "a216a"      "a217a"      "a218a"      "a219a"     
[156] "a222a"      "a223a"      "a224a"      "a225a"      "a226a"     
[161] "a227a"      "a228a"      "a229a"      "a231a"      "a235a"     
[166] "a236a"      "a237a"      "a238a"      "a242a"      "a243a"     
[171] "a244a"      "a245a"      "a247a"      "a248a"      "a252a"     
[176] "a253a"      "a256a"      "a433a"      "a434a"      "a436a"     
[181] "a529a"      "a534a"      "a747a"      "a748a"      "a752a"     
[186] "a764a"      "anio"

tibble [47,412 × 187] (S3: tbl_df/tbl/data.frame)
 $ entidad   : chr [1:47412] "09" "09" "09" "09" ...
 $ municipio : chr [1:47412] NA NA NA NA ...
 $ codigo    : chr [1:47412] NA NA NA NA ...
 $ id_estrato: num [1:47412] NA 1 2 3 4 NA 99 NA 99 NA ...
 $ ue        : num [1:47412] 427959 396040 23622 6441 1856 ...
 $ a111a     : num [1:47412] 4280287 503719 381732 713094 2681741 ...
 $ a121a     : num [1:47412] 1987650 159109 190346 401345 1236851 ...
 $ a131a     : num [1:47412] 2292636 344611 191387 311750 1444889 ...
 $ a211a     : num [1:47412] 247811 5777 4809 22671 214554 ...
 $ a221a     : num [1:47412] 236512 6230 6072 11177 213033 ...
 $ a511a     : num [1:47412] 456421 101958 105791 133129 115543 ...
 $ a700a     : num [1:47412] 6400326 449131 664426 1011875 4274893 ...
 $ a800a     : num [1:47412] 8303585 810731 899561 1314579 5278714 ...
 $ h000a     : num [1:47412] 3222227 878504 392830 431702 1519191 ...
 $ h000b     : num [1:47412] 1880437 484953 224092 261065 910327 ...


In [88]:
#Función para obtener la clasificación de cada actividad
classif_actividades <- map_dfr(.x = lista_censos,
                                 .f = ~ .x |>
    filter(is.na(municipio) & is.na(id_estrato) & !is.na(codigo)) |>
    mutate(codigo = as.character(codigo),
           tipo = case_when(codigo %in% actividades$clave ~ "Economia circular",
                             TRUE ~ "No economia circular")))
                             

classif_actividades|>
  filter(tipo == "Economia circular")|>
  pull(codigo)|>
  unique()

vacb_economia_circular <- classif_actividades|>
          group_by(anio, tipo) |>
          summarise(vacb_p_act = sum(a131a, na.rm = TRUE))|>
          mutate(Porcentaje = round(vacb_p_act * 100 / sum(vacb_p_act), 2))

vacb_economia_circular

[1] "3251"  "43431" "46641" "53211" "53228" "53231" "53242" "562"   "71151"
[10] "81111" "81121" "8114"  "81221"

`summarise()` has grouped output by 'anio'. You can override using the
`.groups` argument.


anio,tipo,vacb_p_act,Porcentaje
<int>,<chr>,<dbl>,<dbl>
2004,Economia circular,-362.624,-0.01
2004,No economia circular,4501252.309,100.01
2009,Economia circular,4771.997,0.07
2009,No economia circular,6444482.770,99.93
2014,Economia circular,5882.814,0.08
2014,No economia circular,7273364.356,99.92
2019,Economia circular,16304.813,0.15
2019,No economia circular,11188484.960,99.85


In [100]:
#PARTICIPACIÓN DE LA ECONOMÍA CIRCULAR
prc_ue_economia_circular <- classif_actividades|>
          group_by(anio, tipo) |>
          summarise(ue_eco_c = sum(ue, na.rm = TRUE))|>
          mutate(Porcentaje = round(ue_eco_c * 100 / sum(ue_eco_c), 2))|> 
          filter(tipo == "Economia circular")

print("Participación de las unidades de economia circular en las unidades economicas")
prc_ue_economia_circular

#Tasa de variación interquinquenal de las unidades econmicas dedicadas a economía circular
tvq_ue_economia_circular <- classif_actividades |>
  group_by(anio, tipo) |>
  summarise(ue_eco_c = sum(ue, na.rm = TRUE), .groups = "drop") |>
  arrange(tipo, anio) |>
  group_by(tipo) |>
  mutate(tasa_var_interquinquenal = (ue_eco_c / lag(ue_eco_c) - 1) * 100)|>
  filter(tipo == "Economia circular")

print("Tasa de variación interquinquenal de las unidades econmicas dedicadas a economía circular")
tvq_ue_economia_circular

`summarise()` has grouped output by 'anio'. You can override using the
`.groups` argument.


[1] "Participación de las unidades de economia circular en las unidades economicas"


anio,tipo,ue_eco_c,Porcentaje
<int>,<chr>,<dbl>,<dbl>
2004,Economia circular,29439,1.79
2009,Economia circular,30855,1.69
2014,Economia circular,33835,1.63
2019,Economia circular,33030,1.54


[1] "Tasa de variación interquinquenal de las unidades econmicas dedicadas a economía circular"


anio,tipo,ue_eco_c,tasa_var_interquinquenal
<int>,<chr>,<dbl>,<dbl>
2004,Economia circular,29439,NA
2009,Economia circular,30855,4.809946
2014,Economia circular,33835,9.658078
2019,Economia circular,33030,-2.379193


In [99]:
##VARIABLES DE INTERES
# H001A = Personal ocupado total: Comprende a todas las personas que trabajaron durante el periodo de referencia dependiendo contractualmente o no de la unidad económica, sujetas a su dirección y control.
#PARTICIPACIÓN DEL PERSONAL OCUPADO EN ECONOMIA CIRCULAR
prc_po_economia_circular <- classif_actividades|>
          group_by(anio, tipo) |>
          summarise(po_eco_c = sum(h001a, na.rm = TRUE))|>
          mutate(Porcentaje = round(po_eco_c * 100 / sum(po_eco_c), 2))|> 
          filter(tipo == "Economia circular")

print("Participación de la población ocupada en economia circular")
prc_po_economia_circular

#Tasa de variación interquinquenal de la población ocupada en economía circular
tvq_po_economia_circular <- classif_actividades |>
  group_by(anio, tipo) |>
  summarise(po_eco_c = sum(h001a, na.rm = TRUE), .groups = "drop") |>
  arrange(tipo, anio) |>
  group_by(tipo) |>
  mutate(tasa_var_interquinquenal = (po_eco_c / lag(po_eco_c) - 1) * 100)|>
  filter(tipo == "Economia circular")

print("Tasa de variación intrquinquenal de la población ocupada en economia circular")
tvq_po_economia_circular

`summarise()` has grouped output by 'anio'. You can override using the
`.groups` argument.


[1] "Participación de la población ocupada en economia circular"


anio,tipo,po_eco_c,Porcentaje
<int>,<chr>,<dbl>,<dbl>
2004,Economia circular,84234,0.63
2009,Economia circular,84829,0.54
2014,Economia circular,86575,0.49
2019,Economia circular,81616,0.39


[1] "Tasa de variación intrquinquenal de la población ocupada en economia circular"


anio,tipo,po_eco_c,tasa_var_interquinquenal
<int>,<chr>,<dbl>,<dbl>
2004,Economia circular,84234,NA
2009,Economia circular,84829,0.7063656
2014,Economia circular,86575,2.0582584
2019,Economia circular,81616,-5.7279815
